In [33]:
import os 
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [3]:
plays = pd.read_csv('plays.csv')
players = pd.read_csv('players.csv')
playerplay = pd.read_csv('player_play.csv')
tracking_data = pd.read_csv('tracking_week_1.csv')

In [4]:
playerposition = players[['nflId','position']]

In [5]:
filteredtrack = tracking_data[['gameId','playId','frameId','nflId','displayName','x','y','event']]

In [6]:
filteredtrack = pd.merge(
    filteredtrack,
    playerposition,
    how='left',
    left_on=['nflId'],
    right_on=['nflId']
)

In [7]:
filteredtrack = filteredtrack[(filteredtrack['position']=="WR") | (filteredtrack['position']=="TE") | (filteredtrack['position']=="RB")]

In [8]:
filteredtrack

,gameId,playId,frameId,nflId,displayName,x,y,event,position
815,2022091200,64,1,42412.0,Tyler Lockett,32.49,22.32,huddle_break_offense,WR
816,2022091200,64,2,42412.0,Tyler Lockett,32.58,22.06,NaN,WR
817,2022091200,64,3,42412.0,Tyler Lockett,32.69,21.79,NaN,WR
818,2022091200,64,4,42412.0,Tyler Lockett,32.81,21.50,NaN,WR
819,2022091200,64,5,42412.0,Tyler Lockett,32.95,21.21,NaN,WR
...,...,...,...,...,...,...,...,...,...
7103995,2022090800,3696,171,53678.0,Ben Skowronek,3.01,50.29,pass_outcome_incomplete,WR
7103996,2022090800,3696,172,53678.0,Ben Skowronek,2.71,50.21,NaN,WR
7103997,2022090800,3696,173,53678.0,Ben Skowronek,2.43,50.12,NaN,WR
7103998,2022090800,3696,174,53678.0,Ben Skowronek,2.17,50.02,NaN,WR


In [9]:
motionplayers = playerplay[playerplay['motionSinceLineset']==True]

In [10]:
motionplayers[['gameId','playId','nflId','motionSinceLineset','inMotionAtBallSnap']]

,gameId,playId,nflId,motionSinceLineset,inMotionAtBallSnap
2,2022090800,56,42489,True,False
28,2022090800,80,47857,True,True
53,2022090800,101,53079,True,True
119,2022090800,191,53079,True,False
138,2022090800,212,47879,True,True
...,...,...,...,...,...
353695,2022110700,2261,43362,True,False
353740,2022110700,2307,43362,True,False
353863,2022110700,2496,44879,True,False
354253,2022110700,3059,54604,True,False


In [11]:
positions = players[['nflId','position']]

In [12]:
motionplayers = pd.merge(
    motionplayers,
    positions,
    how = "left",
    left_on=['nflId'],
    right_on=['nflId']
)

In [13]:
motionplayers = motionplayers[(motionplayers['position']=="WR") | (motionplayers['position']=="TE") | (motionplayers['position']=="RB")]

In [14]:
motionplayers

,gameId,playId,nflId,teamAbbr,hadRushAttempt,rushingYards,hadDropback,passingYards,sackYardsAsOffense,hadPassReception,...,routeRan,blockedPlayerNFLId1,blockedPlayerNFLId2,blockedPlayerNFLId3,pressureAllowedAsBlocker,timeToPressureAllowedAsBlocker,pff_defensiveCoverageAssignment,pff_primaryDefensiveCoverageMatchupNflId,pff_secondaryDefensiveCoverageMatchupNflId,position
0,2022090800,56,42489,BUF,0,0,0,0,0,1,...,IN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WR
1,2022090800,80,47857,BUF,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RB
4,2022090800,212,47879,BUF,0,0,0,0,0,0,...,IN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TE
5,2022090800,236,52536,BUF,0,0,0,0,0,1,...,CORNER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WR
6,2022090800,299,43399,LA,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5816,2022110700,2182,33130,BAL,0,0,0,0,0,0,...,GO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WR
5817,2022110700,2261,43362,BAL,0,0,0,0,0,0,...,FLAT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RB
5818,2022110700,2307,43362,BAL,0,0,0,0,0,0,...,FLAT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RB
5819,2022110700,2496,44879,NO,0,0,0,0,0,0,...,FLAT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RB


In [15]:
motiontracking = pd.merge(
    filteredtrack,
    motionplayers[['gameId','playId','nflId','motionSinceLineset','inMotionAtBallSnap']],
    how="inner",
    left_on=['gameId','playId','nflId'],
    right_on=['gameId','playId','nflId']
)

In [16]:
motiontracking

,gameId,playId,frameId,nflId,displayName,x,y,event,position,motionSinceLineset,inMotionAtBallSnap
0,2022091200,64,1,46189.0,Will Dissly,33.97,25.45,huddle_break_offense,TE,True,False
1,2022091200,64,2,46189.0,Will Dissly,34.01,25.45,NaN,TE,True,False
2,2022091200,64,3,46189.0,Will Dissly,34.04,25.45,NaN,TE,True,False
3,2022091200,64,4,46189.0,Will Dissly,34.08,25.45,NaN,TE,True,False
4,2022091200,64,5,46189.0,Will Dissly,34.11,25.44,NaN,TE,True,False
...,...,...,...,...,...,...,...,...,...,...,...
238782,2022090800,3674,167,43399.0,Tyler Higbee,22.14,49.91,pass_outcome_incomplete,TE,True,True
238783,2022090800,3674,168,43399.0,Tyler Higbee,22.11,49.99,NaN,TE,True,True
238784,2022090800,3674,169,43399.0,Tyler Higbee,22.07,50.04,NaN,TE,True,True
238785,2022090800,3674,170,43399.0,Tyler Higbee,22.02,50.04,NaN,TE,True,True


In [17]:
def filter_motion_to_snap(group):
    group = group.sort_values(by='frameId').reset_index(drop=True)
    line_set_indices = group[group['event'] == 'line_set'].index
    snap_indices = group[group['event'] == 'ball_snap'].index
    
    filtered_rows = []
    for start_idx in line_set_indices:
        snap_idx = snap_indices[snap_indices > start_idx].min()
        if pd.notna(snap_idx):
            filtered_rows.extend(group.loc[start_idx:snap_idx].index)
    return group.loc[filtered_rows]
filtered_df = motiontracking.groupby(['gameId', 'playId']).apply(filter_motion_to_snap).reset_index(drop=True)

In [18]:
filtered_df

,gameId,playId,frameId,nflId,displayName,x,y,event,position,motionSinceLineset,inMotionAtBallSnap
0,2022090800,56,76,42489.0,Stefon Diggs,87.02,27.91,line_set,WR,True,False
1,2022090800,56,77,42489.0,Stefon Diggs,87.04,28.36,NaN,WR,True,False
2,2022090800,56,78,42489.0,Stefon Diggs,87.06,28.82,NaN,WR,True,False
3,2022090800,56,79,42489.0,Stefon Diggs,87.07,29.27,NaN,WR,True,False
4,2022090800,56,80,42489.0,Stefon Diggs,87.09,29.73,NaN,WR,True,False
...,...,...,...,...,...,...,...,...,...,...,...
162169,2022091200,3723,306,52454.0,K.J. Hamler,71.56,39.21,NaN,WR,True,False
162170,2022091200,3723,307,52454.0,K.J. Hamler,71.56,39.21,NaN,WR,True,False
162171,2022091200,3723,308,52454.0,K.J. Hamler,71.56,39.21,NaN,WR,True,False
162172,2022091200,3723,309,52454.0,K.J. Hamler,71.56,39.21,NaN,WR,True,False


In [20]:
filtered_df[['gameId','playId']].drop_duplicates().shape

(1076, 2)

In [99]:
motionstats = filtered_df.groupby(['gameId', 'playId', 'nflId'])['y'].agg(['first', 'last', 'max', 'min']).reset_index()

In [100]:
motionstats

,gameId,playId,nflId,first,last,max,min
0,2022090800,56,42489.0,27.91,43.65,43.67,27.91
1,2022090800,80,47857.0,39.38,31.54,39.41,28.69
2,2022090800,212,47879.0,23.83,32.83,32.83,23.83
3,2022090800,236,52536.0,40.09,35.27,40.09,35.27
4,2022090800,299,43399.0,23.81,18.99,23.81,18.86
...,...,...,...,...,...,...,...
1328,2022091200,3382,46109.0,29.27,33.27,33.27,29.27
1329,2022091200,3491,46096.0,20.99,27.25,27.25,20.99
1330,2022091200,3553,52423.0,33.44,28.83,33.56,28.83
1331,2022091200,3596,52423.0,34.88,16.78,34.88,16.78


In [101]:
motionstats['direction_change'] = np.where((motionstats['last'] > motionstats['first']) & (motionstats['max'] > (motionstats['last'] + 1)),
    True,
    np.where((motionstats['last'] <= motionstats['first']) & (motionstats['min'] < (motionstats['last'] - 1)),True,False)
)


In [102]:
motionstats

,gameId,playId,nflId,first,last,max,min,direction_change
0,2022090800,56,42489.0,27.91,43.65,43.67,27.91,False
1,2022090800,80,47857.0,39.38,31.54,39.41,28.69,True
2,2022090800,212,47879.0,23.83,32.83,32.83,23.83,False
3,2022090800,236,52536.0,40.09,35.27,40.09,35.27,False
4,2022090800,299,43399.0,23.81,18.99,23.81,18.86,False
...,...,...,...,...,...,...,...,...
1328,2022091200,3382,46109.0,29.27,33.27,33.27,29.27,False
1329,2022091200,3491,46096.0,20.99,27.25,27.25,20.99,False
1330,2022091200,3553,52423.0,33.44,28.83,33.56,28.83,False
1331,2022091200,3596,52423.0,34.88,16.78,34.88,16.78,False


In [103]:
motionstats = pd.merge(
    motionstats,
    playerplay[['gameId','playId','nflId','inMotionAtBallSnap']],
    how="left",
    left_on=['gameId','playId','nflId'],
    right_on=['gameId','playId','nflId']
)

In [104]:
motionstats

,gameId,playId,nflId,first,last,max,min,direction_change,inMotionAtBallSnap
0,2022090800,56,42489.0,27.91,43.65,43.67,27.91,False,False
1,2022090800,80,47857.0,39.38,31.54,39.41,28.69,True,True
2,2022090800,212,47879.0,23.83,32.83,32.83,23.83,False,True
3,2022090800,236,52536.0,40.09,35.27,40.09,35.27,False,True
4,2022090800,299,43399.0,23.81,18.99,23.81,18.86,False,False
...,...,...,...,...,...,...,...,...,...
1328,2022091200,3382,46109.0,29.27,33.27,33.27,29.27,False,True
1329,2022091200,3491,46096.0,20.99,27.25,27.25,20.99,False,True
1330,2022091200,3553,52423.0,33.44,28.83,33.56,28.83,False,True
1331,2022091200,3596,52423.0,34.88,16.78,34.88,16.78,False,True


In [105]:
motion = motionstats[['gameId','playId','nflId','first','last','direction_change','inMotionAtBallSnap']]

In [109]:
motionstats[abs(motionstats['first']-motionstats['last']) > 15]

,gameId,playId,nflId,first,last,max,min,direction_change,inMotionAtBallSnap
0,2022090800,56,42489.0,27.91,43.65,43.67,27.91,False,False
8,2022090800,364,44881.0,29.69,14.66,29.75,14.65,False,False
10,2022090800,393,47853.0,8.18,25.85,25.87,8.18,False,False
17,2022090800,721,44985.0,17.39,46.86,46.90,17.39,False,False
44,2022090800,1991,47853.0,7.97,23.02,23.02,7.97,False,False
...,...,...,...,...,...,...,...,...,...
1317,2022091200,2923,47987.0,8.90,31.52,31.52,8.88,False,True
1320,2022091200,2944,53485.0,31.20,15.14,31.20,15.05,False,False
1324,2022091200,3194,46109.0,21.81,38.41,38.43,20.99,False,False
1327,2022091200,3296,52423.0,23.18,39.03,39.03,23.18,False,True
